# Own your health app!

Trying to show a simple PoC where I give it a medical test document and then ask 
some structured responses from it. 

For our real app, we will have 10's or 100's of such documents per individual 
and each document could be 10's to 100's of pages. This is because someone's diagnostic 
journey e.g. cancer is spread across tests and visits to 100's of institutions before they
get refered to a large cancer center oncologist and this oncology team has to make sense of all you have
endured {treatment, outcomes, discharge summaries} to give you the right next treatment when 
you arrive at their doorstep.


For now success will be if for a single report I am able to get back all the
diagnostic tests done without missing. It tends to miss a few and on repeated nudging since 
I know the answer, pull out more and more tests missed previously.
Also, as a bonus what I would really really like would be I ask a question with a schema 
and it returns *all* the elements in the same schema.


In [ ]:
from llama_index import SimpleDirectoryReader
from llama_index.node_parser import SimpleNodeParser
import weaviate

In [ ]:
# connect to your weaviate instance

from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options=EmbeddedOptions()
)


![alternative text](../docs/images/PXL_20230726_203549965.jpg)

## This is a pathology report, containing 7 some pages of all sorts of blood work essential at a particular cancer journey stage

In [ ]:
path_data = SimpleDirectoryReader('../data').load_data()

In [ ]:
# chunk up the data posts into nodes 
parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(path_data)

In [ ]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex, StorageContext
from llama_index.storage.storage_context import StorageContext


# construct vector store
vector_store = WeaviateVectorStore(weaviate_client = client, index_name="Cancer_patient_bloodwork", text_key="content")

# setting up the storage for the embeddings
storage_context = StorageContext.from_defaults(vector_store = vector_store)
#
# set up the index
index = VectorStoreIndex(nodes, storage_context = storage_context)


In [ ]:
# Segmenting report into pre determined types. This can also be done via enums in "class" code.
query_engine = index.as_query_engine()
question = """
What kind of report is this? 
Your choices are pathology report, lab report, genetic test report, clinical notes or radiology report.
"""
report_type = query_engine.query(question)
print(report_type)


In [ ]:
# Segmenting report into pre determined types. This can also be done via enums in "class" code.
query_engine = index.as_query_engine()
question = """
Please give me back a table of all the analytes measured. The table should have following columns: analyte_measured, result, reference_interval, unit, notes. 
If a patricular column does not exist please say NA.
Please double check your work and do not miss any analyte.
"""
response = query_engine.query(str(report_type)+question)
print(response)
# It's only doing the first page!

In [ ]:
# How did it come to the conclusion that it was a patholiogy report?

evidence = response.source_nodes[0].node.text
location = response.source_nodes[0].node.metadata
evidence, location

### Some more harder queries

## Attempt to configure the retriever so I can get *all* results!

## Making it more complex since we need top 'n' results not only top 2

In [ ]:
question = """
Please give me back a table of all the analytes measured. The table should have following columns: analyte_measured, result, reference_interval, unit, test_date, page_number, notes. 
If a patricular column does not exist please say NA.
Please double check your work and do not miss any analyte.
"""

from llama_index import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine


# configure retriever
retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# query
response = query_engine.query(question)
print(response)
# Major issues:
# It is missing a lot of values, sometimes partially parsing pages
# Also, here, I specify k=10, this cannot be pre-configurable and needs to be done every page at a time
# Also it is making up wrong page numbers.

# Minor issues:
# Seems like on this one it is giving NA for entries which are found randomly in text. 
# Do I need better prompting?